<a href="https://colab.research.google.com/github/Jaeji/AM360Paper/blob/main/Model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# ลบโฟลเดอร์เดิมทิ้งก่อน เพื่อป้องกันกรณีไฟล์มาไม่ครบ
!rm -rf AM360Paper

# Clone ข้อมูลมาใหม่
!git clone https://github.com/Jaeji/AM360Paper.git

# เช็คไฟล์ว่ามาครบไหม
import os
print("Checking .mat files:")
for root, dirs, files in os.walk("./AM360Paper"):
    for f in files:
        if f.endswith(".mat"):
            print(f)

Cloning into 'AM360Paper'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 30 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 53.98 MiB | 30.59 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Checking .mat files:
B0007.mat
B0018.mat
B0005.mat
B0006.mat


# Section 1: Import ไลบรารีและเตรียมฟังก์ชันอ่านไฟล์

In [51]:
import numpy as np
import pandas as pd
import os, time
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import warnings

# ปิด Warning เพื่อความสะอาดของ Output
warnings.filterwarnings('ignore')

# ฟังก์ชันค้นหาไฟล์ในโฟลเดอร์
def find_mat_file(filename, search_path='./AM360Paper'):
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            return os.path.join(root, filename)
    return None

# ฟังก์ชันดึง Feature จากไฟล์ .mat
def extract_features(mat_path, battery_name):
    mat = loadmat(mat_path)
    # หา key ที่ถูกต้องในไฟล์ mat
    if battery_name in mat:
        battery = mat[battery_name]
    else:
        keys = [k for k in mat.keys() if not k.startswith('__')]
        battery = mat[keys[0]]

    cycles = battery[0][0]["cycle"][0]
    rows = []
    cycle_id = 1

    for c in cycles:
        # เลือกเฉพาะข้อมูล discharge ที่สมบูรณ์
        if c["type"][0] == "discharge" and "data" in c.dtype.names:
            d = c["data"][0][0]
            if "Capacity" in d.dtype.names and d["Capacity"].size > 0:
                rows.append([
                    battery_name,
                    cycle_id,
                    np.mean(d["Voltage_measured"][0]),
                    np.std(d["Voltage_measured"][0]),
                    np.mean(d["Current_measured"][0]),
                    np.std(d["Current_measured"][0]),
                    np.mean(d["Temperature_measured"][0]),
                    d["Capacity"][0][0]
                ])
                cycle_id += 1

    return pd.DataFrame(
        rows,
        columns=["battery", "cycle", "V_mean", "V_std", "I_mean", "I_std", "Temp_mean", "Capacity"]
    )

# Section 2: โหลดข้อมูลจากไฟล์ .mat

In [52]:
batteries = ["B0005", "B0006", "B0007", "B0018"]
dfs = []

print("Loading data...")
for b in batteries:
    path = find_mat_file(f"{b}.mat")
    if path:
        print(f"- Found {b} at {path}")
        dfs.append(extract_features(path, b))
    else:
        print(f"- Warning: {b}.mat not found")

if not dfs:
    raise FileNotFoundError("ไม่พบไฟล์ข้อมูล .mat กรุณาตรวจสอบโฟลเดอร์ AM360Paper อีกครั้ง")

df = pd.concat(dfs, ignore_index=True)
print(f"Data loaded successfully. Total rows: {len(df)}")

Loading data...
- Found B0005 at ./AM360Paper/B0005.mat
- Found B0006 at ./AM360Paper/B0006.mat
- Found B0007 at ./AM360Paper/B0007.mat
- Found B0018 at ./AM360Paper/B0018.mat
Data loaded successfully. Total rows: 636


# Section 3: การทำความสะอาดข้อมูล (Data Cleaning)

In [53]:
def clean_data(df):
    df_clean = df.copy()

    # 1. จัดการค่า Missing Values (ถ้ามี)
    if df_clean.isnull().sum().sum() > 0:
        print(f"Found {df_clean.isnull().sum().sum()} missing values. Dropping...")
        df_clean = df_clean.dropna()

    # 2. กรอง Outliers ทางกายภาพ (Physical Constraints)
    # เช่น ความจุ (Capacity) ไม่ควรเป็นลบ หรือ อุณหภูมิไม่ควรเกินช่วงที่ระบุ
    # (ค่าเหล่านี้ขึ้นอยู่กับสเปคแบตเตอรี่ ในที่นี้ตั้งไว้กว้างๆ เพื่อกันค่า error)
    df_clean = df_clean[df_clean['Capacity'] > 0]
    df_clean = df_clean[df_clean['Temp_mean'] > 0]

    # 3. กำจัด Outliers ทางสถิติ (Using IQR Method)
    # ทำเฉพาะ Feature ที่เป็น Sensor readings เพื่อตัดค่า noise กระโดด
    sensor_cols = ['V_mean', 'I_mean', 'Temp_mean']

    for col in sensor_cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # กรองเอาเฉพาะข้อมูลที่อยู่ในย่านปกติ
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

    # 4. Smoothing (ลดสัญญาณรบกวน) **สำคัญสำหรับ RUL**
    # ใช้ Rolling Mean เพื่อให้กราฟเสื่อมสภาพ (Degradation) เรียบขึ้น
    # ช่วยให้โมเดลจับเทรนด์ได้ดีกว่าข้อมูลที่กระโดดไปมา
    # (Window=5 หมายถึงเฉลี่ย 5 รอบล่าสุด)
    df_clean['Capacity_Smooth'] = df_clean.groupby('battery')['Capacity'].transform(
        lambda x: x.rolling(window=5, center=True, min_periods=1).mean()
    )

    # อัปเดต Capacity เดิมด้วยค่าที่ Smooth แล้ว (ถ้าต้องการ)
    # หรือจะใช้ Feature ใหม่ 'Capacity_Smooth' ในการเทรนก็ได้
    df_clean['Capacity'] = df_clean['Capacity_Smooth']
    df_clean = df_clean.drop(columns=['Capacity_Smooth'])

    return df_clean

# เรียกใช้งานฟังก์ชัน
print(f"Original Data Size: {len(df)}")
df = clean_data(df)
print(f"Cleaned Data Size: {len(df)}")

# เช็คดูข้อมูลหลังจากคลีน
print(df.describe().round(4))

Original Data Size: 636
Cleaned Data Size: 627
          cycle    V_mean     V_std    I_mean     I_std  Temp_mean  Capacity
count  627.0000  627.0000  627.0000  627.0000  627.0000   627.0000  627.0000
mean    79.5694    3.4998    0.2471   -1.8383    0.5173    32.3090    1.5875
std     46.3974    0.0468    0.0168    0.0970    0.1696     0.9654    0.1934
min      1.0000    3.4039    0.2171   -1.9998    0.1470    30.1027    1.2059
25%     40.0000    3.4679    0.2336   -1.9133    0.4161    31.5124    1.4263
50%     79.0000    3.5042    0.2438   -1.8539    0.5219    32.3522    1.5627
75%    118.0000    3.5412    0.2582   -1.7767    0.6433    33.1335    1.7643
max    168.0000    3.5736    0.2922   -1.5671    0.8329    34.4905    2.0246


# Section 4: คำนวณ RUL และเตรียมข้อมูล Train/Test

In [54]:
# 1. คำนวณ RUL (Remaining Useful Life) หน่วยเป็นจำนวนรอบ (Cycles)
df["RUL"] = df.groupby("battery")["cycle"].transform(lambda x: x.max() - x)

# 2. แบ่ง Train/Test (ตาม Paper ใช้ B0018 เป็น Test set)
train_df = df[df["battery"] != "B0018"]
test_df  = df[df["battery"] == "B0018"]

# Fallback: กรณีไฟล์ B0018 หายไป ให้แบ่งท้าย B0005 มาเทสแทน
if len(test_df) == 0:
    print("Note: B0018 not found. Using part of B0005 for testing.")
    test_df = train_df[train_df["battery"] == "B0005"].tail(50)
    train_df = train_df.drop(test_df.index)

# 3. เตรียม X, y และทำการ Scaling
features = ["V_mean", "V_std", "I_mean", "I_std", "Temp_mean", "Capacity"]
X_train = train_df[features].values
y_train = train_df["RUL"].values
X_test  = test_df[features].values
y_test  = test_df["RUL"].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

Train set: (495, 6), Test set: (132, 6)


# Section 5: กำหนด Model และ Hyperparameters

In [55]:
models = {
    "Lasso": Lasso(alpha=0.1),
    "Random_Forest": RandomForestRegressor(n_estimators=100, max_depth=12, random_state=42),
    "Gradient_Boosting": GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42),
    "Kernel_Ridge": KernelRidge(alpha=1.0, kernel="rbf", gamma=0.01),
    "Decision_Tree": DecisionTreeRegressor(max_depth=8, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, objective="reg:squarederror", random_state=42)
}

# Section 6: เทรนโมเดลและคำนวณค่า Error

In [56]:
# เตรียม List เก็บผลลัพธ์
results_table2 = [] # RMSE, SSE
results_table3 = [] # Total Error
results_table4 = [] # Relative Error
results_table5 = [] # Time

# ลำดับโมเดลตามตารางในรูปภาพ
model_order = ["Lasso", "Random_Forest", "Gradient_Boosting", "Kernel_Ridge", "Decision_Tree", "XGBoost"]

print("Starting Training & Evaluation...")

for name in model_order:
    model = models[name]

    # จับเวลาและเทรน
    start_time = time.time()
    model.fit(X_train, y_train)
    run_time = time.time() - start_time

    # ทำนายผล
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # --- คำนวณ Metrics ---

    # RMSE & SSE
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    sse_train = np.sum((y_train - y_train_pred) ** 2)
    sse_test = np.sum((y_test - y_test_pred) ** 2)

    # Total Error (Sum Absolute Error)
    total_err_train = np.sum(np.abs(y_train - y_train_pred))
    total_err_test = np.sum(np.abs(y_test - y_test_pred))

    # Relative Error (ระวังการหารด้วย 0)
    mask_train, mask_test = y_train > 0, y_test > 0
    rel_err_train = np.mean(np.abs(y_train[mask_train] - y_train_pred[mask_train]) / y_train[mask_train])
    rel_err_test = np.mean(np.abs(y_test[mask_test] - y_test_pred[mask_test]) / y_test[mask_test])

    # บันทึกผล
    results_table2.append([name, rmse_train, rmse_test, sse_train, sse_test])
    results_table3.append([name, total_err_train, total_err_test])
    results_table4.append([name, rel_err_train, rel_err_test])
    results_table5.append([name, run_time])

    print(f"Finished: {name}")

Starting Training & Evaluation...
Finished: Lasso
Finished: Random_Forest
Finished: Gradient_Boosting
Finished: Kernel_Ridge
Finished: Decision_Tree
Finished: XGBoost


# Section 7: แสดงผลลัพธ์ตาราง (Table II - V)

In [57]:
from IPython.display import display

# Table II: RMSE & SSE
print("\nTABLE II: RMSE AND SSE VALUES")
df_table2 = pd.DataFrame(results_table2, columns=["Methods", "RMSE Train", "RMSE Test", "SSE Train", "SSE Test"])
display(df_table2.round(2))

# Table III: Total Error
print("\nTABLE III: TOTAL ERROR VALUES")
df_table3 = pd.DataFrame(results_table3, columns=["Methods", "Total Error Train", "Total Error Test"])
display(df_table3.round(2))

# Table IV: Relative Error
print("\nTABLE IV: AVERAGE RELATIVE ERROR VALUES")
df_table4 = pd.DataFrame(results_table4, columns=["Methods", "Relative Error Train", "Relative Error Test"])
display(df_table4.round(4))

# Table V: Running Time
print("\nTABLE V: RUNNING TIMES")
df_table5 = pd.DataFrame(results_table5, columns=["Methods", "Time (second)"])
display(df_table5.round(4))


TABLE II: RMSE AND SSE VALUES


,Methods,RMSE Train,RMSE Test,SSE Train,SSE Test
0,Lasso,8.55,9.93,36213.12,13019.63
1,Random_Forest,1.52,14.40,1146.09,27374.01
2,Gradient_Boosting,1.43,13.75,1014.47,24939.61
3,Kernel_Ridge,12.59,15.32,78517.74,30983.87
4,Decision_Tree,1.50,19.67,1116.63,51076.10
5,XGBoost,0.16,14.86,12.68,29133.69



TABLE III: TOTAL ERROR VALUES


,Methods,Total Error Train,Total Error Test
0,Lasso,3266.99,1118.53
1,Random_Forest,534.38,1560.08
2,Gradient_Boosting,547.54,1506.99
3,Kernel_Ridge,4401.94,1665.99
4,Decision_Tree,436.02,2181.40
5,XGBoost,56.92,1528.70



TABLE IV: AVERAGE RELATIVE ERROR VALUES


,Methods,Relative Error Train,Relative Error Test
0,Lasso,0.3306,0.5050
1,Random_Forest,0.0372,0.4694
2,Gradient_Boosting,0.0370,0.4890
3,Kernel_Ridge,0.4905,0.9204
4,Decision_Tree,0.0351,0.4475
5,XGBoost,0.0032,0.3546



TABLE V: RUNNING TIMES


,Methods,Time (second)
0,Lasso,0.0019
1,Random_Forest,0.3829
2,Gradient_Boosting,0.5192
3,Kernel_Ridge,0.1276
4,Decision_Tree,0.0120
5,XGBoost,0.6527
